### Implementing improved pre-processing

In [1]:
import os
import pandas
import numpy
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Average, Input, Lambda
from tqdm import tqdm
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from random import randint
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
dataDir = '/home/joel/Documents/KaggleComps/SeedlingId/'
trainDir = os.path.join(dataDir, 'train')
valDir = os.path.join(dataDir, 'validate')
testDir = os.path.join(dataDir, 'test')
sampleSub = pandas.read_csv(os.path.join(dataDir, 'sample_submission.csv'))

In [3]:
categories = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
              'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']
weeds = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Fat Hen', 'Loose Silky-bent',
         'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill']
crops = ['Common wheat', 'Maize', 'Sugar beet']

In [7]:
trainGenerator=image.ImageDataGenerator(
        shear_range=0.2, zoom_range=0.2,
        horizontal_flip=True)
testGenerator=image.ImageDataGenerator()

In [12]:
size = (250,250,3)

model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(size))) # Added Normalization and centering.
model.add(Conv2D(16, 3, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(12))
model.add(Activation('softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_5 (Lambda)            (None, 250, 250, 3)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 250, 250, 16)      448       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 125, 125, 16)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 125, 125, 32)      4640      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 62, 62, 64)        18496     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 31, 31, 64)        0         
__________

In [ ]:
checkpointer = ModelCheckpoint(filepath='weights/rgbNorm{}.hdf5'.format(size[0]), verbose=1, save_best_only=True)

model.fit_generator(trainGenerator.flow_from_directory(trainDir, target_size=size[:2], batch_size=20),
                    epochs=20, validation_data=testGenerator.flow_from_directory(valDir, target_size=size[:2],
                                                                                 batch_size=20),
                    callbacks=[checkpointer], verbose=1)

Benchmark CNN validation accuracy improved from 75% to 80%.

In [ ]:
size = (299,299,3)
inp = Input(size)
norm = Lambda(lambda x: (x / 255.0) - 0.5)(inp)  # Added normalization and centering.
IRmodel = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=norm)

for layer in IRmodel.layers[:618]:
   layer.trainable = False
for layer in IRmodel.layers[618:]:
   layer.trainable = True

x=IRmodel.output
x = GlobalAveragePooling2D()(x)
x = Dense(122, activation='tanh')(x)
x = Dropout(.5)(x)
predictions = Dense(12, activation='softmax')(x)

IRmodel = Model(inputs=inp, outputs=predictions)

IRmodel.compile(optimizer=SGD(lr=0.0005, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='weights/IRmodelNORM{}.hdf5'.format(size[0]), verbose=1, save_best_only=True)

IRmodel.fit_generator(trainGenerator.flow_from_directory(trainDir, target_size=size[:2], batch_size=20), epochs=20,
                      validation_data=testGenerator.flow_from_directory(valDir, target_size=size[:2], batch_size=20),
                      callbacks=[checkpointer], verbose=1)